In [2]:
import matplotlib.pyplot as plt
from astropy.time import Time
from sbpy.data import Orbit

In [3]:
obs_dates = {
    'spitzer': Time(['2007-12-08', '2008-01-17', '2008-04-24', 
                     '2008-05-24', '2008-07-02']),
    'bass': Time('2018-12-10')
}

In [17]:
comet = Orbit.from_horizons(
        '46P', 'designation', epochs=(obs_dates['spitzer'].jd.mean(), obs_dates['bass']),
        closest_apparition=True)
planets = Orbit.from_horizons((399, 599), 'majorbody', epochs=obs_dates['bass'])